In [1]:
import qctoolkit as qtk
from matplotlib import pyplot as plt
import matplotlib as mpl
import numpy as np
from copy import deepcopy
import horton as ht
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import UnivariateSpline
from glob import glob
import os
from scipy.interpolate import spline
from scipy.interpolate import interp2d

#Note inline and notebook is contradicting to each other
%matplotlib inline

/home/samio/local/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


missing file: mov2asc
please modify /path/to/qctoolkit/setting.py and recompile.


In [4]:
def list2mesh(data=None, E=None, X=None):
    if data is not None:
        X = data['X']
        E = data['E']
    X_lists = [sorted(set(X[:,i])) for i in range(X.shape[1])]
    X_meshes = np.array(np.meshgrid(*X_lists, indexing='ij'))
    E_mesh = np.zeros(X_meshes[0].shape)
    for s, e in enumerate(E):
        inds = tuple([np.argmin(abs(np.array(arr) - X[s][i])) for i, arr in enumerate(X_lists)])
        E_mesh[inds] = e
    return X_meshes, E_mesh

In [2]:
data_pbe_HAl = np.load('gaussian_N2_pbe_H-Al.npz')
En = data_pbe_HAl['E_neutral']
Xn = data_pbe_HAl['X_neutral']
dx = data_pbe_HAl['dx']
g_list, h_list = data_pbe_HAl['g_list'][:,:2], data_pbe_HAl['h_list'][:,:2,:2]

In [5]:
Xn_meshes, E_mesh = list2mesh(E=En, X=Xn)

In [6]:
for h in h_list:
    w, v = np.linalg.eigh(h)
    print v[:,0], v[:,1]

[ 0.70710678 -0.70710678] [0.70710678 0.70710678]
[-0.44225225  0.89689071] [-0.89689071 -0.44225225]
[-0.30590564  0.95206184] [-0.95206184 -0.30590564]
[-0.22475245  0.9744159 ] [-0.9744159  -0.22475245]
[-0.89033775  0.45530066] [-0.45530066 -0.89033775]
[-0.92631367  0.37675321] [-0.37675321 -0.92631367]
[-0.82103319  0.57088046] [-0.57088046 -0.82103319]


In [11]:
def get_zz_coord(X_meshes, dz=0, dR=None):
    dZ = X_meshes[1][0,:,0] - (7-dz)
    Z_coord = np.asarray(zip(dZ, (dZ-2*dz)[::-1]))
    if dR is None:
        return Z_coord
    else:
        return np.hstack([Z_coord, (np.ones(len(Z_coord)) * dR)[:,np.newaxis]])
    
def get_zz_coord_eig(X_meshes, h_list, dz=0):
    Z_coord = get_zz_coord(X_meshes, dz)
    w, v = np.linalg.eigh(h_list[dz])
    return Z_coord.dot(v), v, w

def get_zz_1st(X_meshes, )


(array([[ 6.6957148 ,  2.27319231],
        [ 5.35657184,  1.81855385],
        [ 4.01742888,  1.36391539],
        [ 2.67828592,  0.90927692],
        [ 1.33914296,  0.45463846],
        [ 0.        ,  0.        ],
        [-1.33914296, -0.45463846],
        [-2.67828592, -0.90927692],
        [-4.01742888, -1.36391539],
        [-5.35657184, -1.81855385],
        [-6.6957148 , -2.27319231],
        [-8.03485776, -2.72783077],
        [-9.37400072, -3.18246924]]), array([[-0.44225225, -0.89689071],
        [ 0.89689071, -0.44225225]]), array([-2.51111789, -1.27628211]))

In [13]:
Z_coord_eig, v, w = get_zz_coord_eig(Xn_meshes, h_list, 1)

In [17]:
get_zz_coord_eig(Xn_meshes, h_list, 0)

(array([[-8.48528137,  0.        ],
        [-7.07106781,  0.        ],
        [-5.65685425,  0.        ],
        [-4.24264069,  0.        ],
        [-2.82842712,  0.        ],
        [-1.41421356,  0.        ],
        [ 0.        ,  0.        ],
        [ 1.41421356,  0.        ],
        [ 2.82842712,  0.        ],
        [ 4.24264069,  0.        ],
        [ 5.65685425,  0.        ],
        [ 7.07106781,  0.        ],
        [ 8.48528137,  0.        ]]), array([[ 0.70710678,  0.70710678],
        [-0.70710678,  0.70710678]]), array([-2.38533, -1.36979]))

In [18]:
print Z_coord_eig[4].dot(v.T)
print v

[-1.  1.]
[[-0.44225225 -0.89689071]
 [ 0.89689071 -0.44225225]]


In [26]:
Z_coord_eig[4].dot(v.T).dot(g_list[1])

-7.539634999999972

In [24]:
print np.array([-1,1]).dot(g_list[1]) /
print np.array([1,-1]).dot(g_list[1])

-7.539634999999976
7.539634999999976


In [38]:
for i in range(len(g_list)):
    print np.arctan2(np.array([-1,1]).dot(g_list[i]), 1) * 180 / np.pi

0.0
-82.44481841922031
-86.18915153707468
-87.44297621459033
-88.06799877546989
-88.44030353064487
-88.68014645033816


In [33]:
np.arctan2?